In [9]:
import requests
import lxml.html
import bs4

import numpy as np
import pandas as pd

In [10]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
LOGIN_URL = 'https://www.placement.iitbhu.ac.in/accounts/login'
COMPANY_URL = 'https://www.placement.iitbhu.ac.in/company/calendar'
USERNAME = 'sartaj.singh.apm13@itbhu.ac.in'
PASSWORD = 'Superkid@123'

In [12]:
def make_login_form(session, username, password):
    login = session.get(LOGIN_URL)
    login_html = lxml.html.fromstring(login.text)
    hidden_inputs = login_html.xpath(r'//form//input[@type="hidden"]')
    form = {x.attrib['name']: x.attrib['value'] for x in hidden_inputs}
    form['login'] = username
    form['password'] = password
    return form

In [13]:
session = requests.session()

In [14]:
form = make_login_form(session, USERNAME, PASSWORD)
form

{'csrfmiddlewaretoken': '7ECAVHlxVbwFEY2sJBLubRlCTS9rKv3Q',
 'login': 'sartaj.singh.apm13@itbhu.ac.in',
 'password': 'Superkid@123'}

In [15]:
params = {'Referer': LOGIN_URL, 'User-Agent': USER_AGENT}
login = session.post(LOGIN_URL, data=form, headers=params)
login

<Response [200]>

In [16]:
params = {'page_size': 250}
data = session.get(COMPANY_URL, params=params)
data

<Response [200]>

In [17]:
company_data = bs4.BeautifulSoup(data.text, 'html.parser')

In [ ]:
data_mapper = {
    'updated_at': 'updated_at',
    'company_name': 'company_name',
    'profile': 'company_profile',
    'purpose': 'purpose',
    'x_percent': 'x',
    'xii_percent': 'xii',
    'cgpa': 'cgpa',
    'course': 'course',
    'dept': 'dept',
    'active_backlog': 'a_backlog',
    'total_backlog': 't_backlog',
    'ppt_date': 'ppt_date',
    'exam_date': 'exam_date',
    'interview_date': 'interview_date',
    'status': 'status',
    'branch_issue_deadline': 'branch_issue_dead',
    'willingness_deadline': 'willingness_dead'    
}

In [ ]:
clips = {
    'active_backlog': 'Active backlog allowed :'.find(':') ,
    'branch_issue_deadline': 'Branch Issues Deadline :'.find(':') + 1,
    'cgpa': 'cgpa :'.find(':'),
    'course': 'Course(s) :'.find(':'),
    'dept': 'Department(s) :'.find(':'),
    'exam_date': 'Exam Date :'.find(':'),
    'interview_date': 'Interview Date :'.find(':') ,
    'ppt_date': 'PPT Date : '.find(':'),
    'status': 'Status : '.find(':'),
    'total_backlog': 'Total backlog allowed :'.find(':'),  
    'updated_at': 'st Updated:'.find(':') + 2,
    'willingness_deadline': 'Willingness Deadline :'.find(':') + 1,
    'x_percent': 'X :'.find(':'),
    'xii_percent': 'XII :'.find(':')
}

In [142]:
def get_package_data(company_div, company):
    package = company_div.find('div', attrs={'class': 'package'})
    for tr in package.find_all('tr'):
        course, num = False, 0
        course_name = None
        for td in tr.find_all('td'):
            if course:
                if num == 0 and td.text:
                    company['{}_ctc'.format(course_name)] = td.text
                if num == 1 and td.text:
                    company['{}_home'.format(course_name)] = td.text
                if num == 1:
                    course = False
                num += 1
            if td.text.lower() in ['b.tech', 'idd/imd', 'm.tech', 'phd']:
                course, num = True, 0
                course_name = td.text.lower().replace('/', '_').replace('.', '')

In [143]:
def get_data(data_mapper):
    companies = []
    for company_div in company_data.find_all('div', attrs={'class': 'row company'}):
        company = {}
        for name, class_name in data_mapper.items():
            company[name] = company_div.find(attrs={'class': class_name}).text
        get_package_data(company_div, company)
        companies.append(company)
    return companies

In [ ]:
def clip_data(companies, clips):
    for company in companies:
        for clip_name, clip_num in clips.items():
            num = clip_num + 2
            company[clip_name] = company[clip_name][num:]
    return companies

In [ ]:
def find_depts(df):
    depts = set([])
    for value in df.dept.unique():
        for dept in value.split(' '):
            depts.add(dept)
    return depts

In [187]:
def process_df(companies):
    df = pd.DataFrame(companies)
    df.loc[:, 'active_backlog'] = df['active_backlog'].replace({'N/A': 0}).astype(np.float64)
    df.loc[:, 'total_backlog'] = df['total_backlog'].replace({'N/A': 0}).astype(np.float64)
    df.loc[:, 'cgpa'] = df['cgpa'].astype(np.float64)
    df.loc[:, 'exam_date'] = pd.to_datetime(df['exam_date'])
    df.loc[:, 'interview_date'] = pd.to_datetime(df['interview_date'])
    df.loc[:, 'ppt_date'] = pd.to_datetime(df['ppt_date'])
    df.loc[:, 'branch_issue_deadline'] = pd.to_datetime(df['branch_issue_deadline'].str.replace('|', ','))
    df.loc[:, 'updated_at'] = pd.to_datetime(df['updated_at'])
    df.loc[:, 'x_percent'] = df['x_percent'].str[:-1].astype(np.float64)
    df.loc[:, 'xii_percent'] = df['xii_percent'].str[:-1].astype(np.float64)
    df.loc[:, 'profile'] = df['profile'].replace({'': np.nan})
    df.loc[:, 'willingness_deadline'] = pd.to_datetime(df['willingness_deadline'].str.replace('|', ','))
    
    for dept in find_depts(df):
        df.loc[:, dept] = df.dept.apply(lambda x: 1 if dept in x else 0)
    
    for course in ['btech', 'idd', 'mtech', 'phd']:
        df.loc[:, course] = df.course.apply(lambda x: 1 if course in x else 0)

    df = df.drop(['dept', 'course', 'purpose'], axis=1)
    return df

In [188]:
companies = get_data(data_mapper)
clip_data(companies, clips)
df = process_df(companies)
df

,active_backlog,branch_issue_deadline,btech_ctc,btech_home,cgpa,company_name,exam_date,idd_imd_ctc,idd_imd_home,interview_date,...,met,mst,civ,cer,mat,chy,btech,idd,mtech,phd
0,0.0,2017-09-06 12:00:00,NaN,NaN,7.0,INTEL,NaT,"17,09,140","10,86,000",NaT,...,0,0,0,0,0,0,0,0,1,0
1,0.0,2017-09-20 09:00:00,NaN,NaN,0.0,DR REDDYS,NaT,NaN,NaN,NaT,...,0,0,0,0,0,0,0,0,1,0
2,0.0,2017-09-21 10:00:00,"8,40,000",NaN,0.0,Country Garden,NaT,"11,20,000",NaN,NaT,...,1,1,1,1,1,1,1,1,0,0
3,0.0,2017-09-20 10:00:00,"27,00,000","12,00,000",0.0,Amazon,2017-10-28,"27,50,000","12,50,000",NaT,...,0,0,0,0,1,0,1,1,0,0
4,0.0,2017-09-06 12:00:00,"22,02.108",NaN,7.0,VMWare,2017-10-16,"22,02.108",NaN,NaT,...,0,0,0,0,0,0,1,0,0,0
5,0.0,2017-09-20 10:00:00,NaN,NaN,7.0,Deutsche Bank,NaT,NaN,NaN,NaT,...,0,0,0,0,0,0,0,0,1,0
6,0.0,2017-09-06 12:00:00,10 to 13 Lakhs,NaN,6.0,ACCOLITE,NaT,10 to 13 Lakhs,NaN,NaT,...,0,0,0,0,1,0,1,1,0,0
7,0.0,2017-09-20 10:00:00,"8,50,000",NaN,6.0,I Cube Nanotec,NaT,NaN,NaN,NaT,...,0,0,0,1,0,0,1,0,0,0
8,0.0,2017-09-20 10:00:00,"10,11,000",NaN,6.5,Tata Steel,NaT,NaN,NaN,NaT,...,0,0,0,1,0,0,1,0,0,0
9,0.0,2017-09-20 10:00:00,"8,00,000",NaN,0.0,CAPGEMINI,NaT,"8,00,000",NaN,NaT,...,1,1,1,1,1,0,1,1,0,0


In [246]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 42 columns):
active_backlog           81 non-null float64
branch_issue_deadline    77 non-null datetime64[ns]
btech_ctc                69 non-null object
btech_home               32 non-null object
cgpa                     81 non-null float64
company_name             81 non-null object
exam_date                29 non-null datetime64[ns]
idd_imd_ctc              65 non-null object
idd_imd_home             35 non-null object
interview_date           0 non-null datetime64[ns]
mtech_ctc                27 non-null object
mtech_home               14 non-null object
phd_ctc                  4 non-null object
phd_home                 2 non-null object
ppt_date                 30 non-null datetime64[ns]
profile                  65 non-null object
status                   81 non-null object
total_backlog            81 non-null float64
updated_at               81 non-null datetime64[ns]
willingness_deadline 

In [247]:
df.to_csv('companies.csv', index=False)
pd.read_csv('companies.csv')

,active_backlog,branch_issue_deadline,btech_ctc,btech_home,cgpa,company_name,exam_date,idd_imd_ctc,idd_imd_home,interview_date,...,met,mst,civ,cer,mat,chy,btech,idd,mtech,phd
0,0.0,2017-09-06 12:00:00,NaN,NaN,7.0,INTEL,NaN,"17,09,140","10,86,000",NaN,...,0,0,0,0,0,0,0,0,1,0
1,0.0,2017-09-20 09:00:00,NaN,NaN,0.0,DR REDDYS,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
2,0.0,2017-09-21 10:00:00,"8,40,000",NaN,0.0,Country Garden,NaN,"11,20,000",NaN,NaN,...,1,1,1,1,1,1,1,1,0,0
3,0.0,2017-09-20 10:00:00,"27,00,000","12,00,000",0.0,Amazon,2017-10-28 00:00:00,"27,50,000","12,50,000",NaN,...,0,0,0,0,1,0,1,1,0,0
4,0.0,2017-09-06 12:00:00,"22,02.108",NaN,7.0,VMWare,2017-10-16 00:00:00,"22,02.108",NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
5,0.0,2017-09-20 10:00:00,NaN,NaN,7.0,Deutsche Bank,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
6,0.0,2017-09-06 12:00:00,10 to 13 Lakhs,NaN,6.0,ACCOLITE,NaN,10 to 13 Lakhs,NaN,NaN,...,0,0,0,0,1,0,1,1,0,0
7,0.0,2017-09-20 10:00:00,"8,50,000",NaN,6.0,I Cube Nanotec,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,1,0,0,0
8,0.0,2017-09-20 10:00:00,"10,11,000",NaN,6.5,Tata Steel,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,1,0,0,0
9,0.0,2017-09-20 10:00:00,"8,00,000",NaN,0.0,CAPGEMINI,NaN,"8,00,000",NaN,NaN,...,1,1,1,1,1,0,1,1,0,0
